Accelerometer Matrix Profile Example
====

In [ ]:
# Read in accelerometer and meal time data from one user

In [ ]:
# Slice the time series to only keep data near a meal event

In [ ]:
# Build the matrix profile for these events

In [ ]:
# Plot them